In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.width", 200)

def repeated_k_fold(model, X, y, n_splits=5, n_repeats=10):
    rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

    accuracy_train, accuracy_test = [], []
    f1, recall, roc_auc = [], [], []

    for train_index, test_index in rkf.split(X, y):
        # Aufteilen in Trainings- und Testdaten
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Skalierung: nur an den Trainingsdaten fitten
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model.fit(X_train_scaled, y_train)

        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)

        accuracy_train.append(accuracy_score(y_train, y_train_pred))
        accuracy_test.append(accuracy_score(y_test, y_test_pred))
        f1.append(f1_score(y_test, y_test_pred))
        recall.append(recall_score(y_test, y_test_pred))
        roc_auc.append(roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1]))

    return {
        "Train Accuracy": (np.mean(accuracy_train), np.std(accuracy_train)),
        "Test Accuracy": (np.mean(accuracy_test), np.std(accuracy_test)),
        "F1-Score_1": (np.mean(f1), np.std(f1)),
        "Recall_1": (np.mean(recall), np.std(recall)),
        "ROC-AUC_1": (np.mean(roc_auc), np.std(roc_auc)),
    }


# Pfad zur Datei
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Basistabelle_ML_neu.xlsx"

try: 
    df = pd.read_excel(file_path)
    
    # Zielvariable (y) und Features (X) extrahieren
    y = df['Verletzungsstatus']
    
    # Dummy-Variable "Geschlecht_weiblich" separieren, falls vorhanden
    if 'Geschlecht_weiblich' in df.columns:
        geschlecht_weiblich = df[['Geschlecht_weiblich']]
        X = df.drop(columns=['Verletzungsstatus', 'Geschlecht_weiblich'])
        # Die Dummy-Variable wieder anhängen
        X = np.hstack((X.values, geschlecht_weiblich.values))
    else:
        X = df.drop(columns=['Verletzungsstatus']).values
    
    # Modelle definieren
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
        "LightGBM": LGBMClassifier(verbose=-1, random_state=42),
        "SVC": SVC(probability=True, random_state=42),
        "k-Nearest Neighbors": KNeighborsClassifier(),
        "MLP Classifier": MLPClassifier(max_iter=1000, random_state=42),
        "Gaussian Naive Bayes": GaussianNB(),
        "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
        "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
        "Bagging Classifier": BaggingClassifier(random_state=42),
        "Extra Trees": ExtraTreesClassifier(random_state=42),
    }
    
    results = []
    for model_name, model in models.items():
        print(f"Modell wird validiert: {model_name}")
        metrics = repeated_k_fold(model, X, y)
        
        formatted_metrics = {
            "Model": model_name,
            "Train Accuracy": f"{metrics['Train Accuracy'][0]:.4f} ± {metrics['Train Accuracy'][1]:.4f}",
            "Test Accuracy": f"{metrics['Test Accuracy'][0]:.4f} ± {metrics['Test Accuracy'][1]:.4f}",
            "F1-Score_1": f"{metrics['F1-Score_1'][0]:.4f} ± {metrics['F1-Score_1'][1]:.4f}",
            "Recall_1": f"{metrics['Recall_1'][0]:.4f} ± {metrics['Recall_1'][1]:.4f}",
            "ROC-AUC_1": f"{metrics['ROC-AUC_1'][0]:.4f} ± {metrics['ROC-AUC_1'][1]:.4f}",
        }
        results.append(formatted_metrics)
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by="ROC-AUC_1", ascending=False)
    
    print("\nErgebnisse der Modelle:")
    print(results_df)


except FileNotFoundError:
    print("Die Datei wurde nicht gefunden. Bitte überprüfe den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Modell wird validiert: Logistic Regression
Modell wird validiert: Decision Tree
Modell wird validiert: Random Forest
Modell wird validiert: Gradient Boosting
Modell wird validiert: XGBoost
Modell wird validiert: LightGBM
Modell wird validiert: SVC
Modell wird validiert: k-Nearest Neighbors
Modell wird validiert: MLP Classifier
Modell wird validiert: Gaussian Naive Bayes
Modell wird validiert: Linear Discriminant Analysis
Modell wird validiert: Quadratic Discriminant Analysis


C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Varia

Modell wird validiert: Bagging Classifier
Modell wird validiert: Extra Trees

Ergebnisse der Modelle:
                              Model   Train Accuracy    Test Accuracy       F1-Score_1         Recall_1        ROC-AUC_1
0               Logistic Regression  1.0000 ± 0.0000  0.8895 ± 0.0572  0.8953 ± 0.0589  0.8914 ± 0.1000  0.9541 ± 0.0461
8                    MLP Classifier  1.0000 ± 0.0000  0.8604 ± 0.0590  0.8651 ± 0.0641  0.8458 ± 0.1053  0.9441 ± 0.0471
4                           XGBoost  1.0000 ± 0.0000  0.8416 ± 0.0656  0.8454 ± 0.0701  0.8162 ± 0.1068  0.9328 ± 0.0376
6                               SVC  0.9777 ± 0.0090  0.8381 ± 0.0692  0.8502 ± 0.0653  0.8559 ± 0.0961  0.9325 ± 0.0454
13                      Extra Trees  1.0000 ± 0.0000  0.8153 ± 0.0775  0.8263 ± 0.0768  0.8214 ± 0.1040  0.9268 ± 0.0523
2                     Random Forest  1.0000 ± 0.0000  0.8180 ± 0.0746  0.8291 ± 0.0713  0.8232 ± 0.0997  0.9209 ± 0.0486
3                 Gradient Boosting  1.0000 ± 0.000

Datenaugmentation (Datenleakage gänzlich vermeiden)

In [5]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform


def perform_clustering(X_sub, max_clusters=4):
    """
    Führt eine Clusteranalyse auf den numerischen Parametern (außer den
    Gruppierungsvariablen "Verletzungsstatus" und "Geschlecht_weiblich")
    durch und teilt diese in max_clusters Gruppen ein.
    """
    # Wähle alle numerischen Spalten außer "Verletzungsstatus" und "Geschlecht_weiblich"
    cols = [col for col in X_sub.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]
    X_num = X_sub[cols]
    
    # Berechne die Korrelationsmatrix und transformiere sie in eine Distanzmatrix
    corr = X_num.corr().abs()
    dist = np.clip(1 - corr, a_min=0, a_max=None)  # Clippe negative Werte auf 0
    
    # Konvertiere die Distanzmatrix in ein 1D-Array (upper triangular)
    dists = squareform(dist.values)
    
    # Hierarchische Clusteranalyse (z. B. mit der Ward-Methode)
    Z = linkage(dists, method='ward')
    
    # Führe fcluster aus, um die Spalten in max_clusters Cluster zu unterteilen
    cluster_labels = fcluster(Z, max_clusters, criterion='maxclust')
    
    # Ordne den Spalten Clusterlabels zu
    clusters = {}
    for col, label in zip(cols, cluster_labels):
        clusters.setdefault(label, []).append(col)
    return clusters


def augment_subgroup(X_sub, clusters, p_augment=0.7):
    """
    Augmentiert die Daten in X_sub (einen DataFrame für eine Subgruppe)
    für alle in clusters definierten Parametergruppen.
    
    Für jede Zeile und für jede Cluster-Gruppe wird mit Wahrscheinlichkeit p_augment
    ein neuer Wert gezogen – ansonsten wird der Originalwert beibehalten.
    Dabei werden für jede Parametergruppe der Mittelwertvektor und die Kovarianzmatrix
    des gesamten Subdatensatzes berechnet.
    
    Rückgabe: DataFrame mit augmentierten Zeilen (1 augmentiertes Sample pro Originalzeile)
    """
    augmented_rows = []
    
    # Für jede Cluster-Gruppe: Berechne Mittelwert und Kovarianzmatrix
    cluster_params = {}
    for cl_id, cols in clusters.items():
        cluster_data = X_sub[cols]
        mu = cluster_data.mean().values
        # Falls nur ein Feature im Cluster ist, gibt np.cov einen Skalar zurück.
        if len(cols) == 1:
            cov = np.cov(cluster_data.values.flatten(), ddof=0)
            cov = np.atleast_2d(cov)
        else:
            cov = np.cov(cluster_data.values, rowvar=False)
        cluster_params[cl_id] = (mu, cov, cols)
    
    # Gehe jede Zeile (Originaldatensatz) durch
    for idx, row in X_sub.iterrows():
        new_row = row.copy()
        # Für jede Cluster-Gruppe stochastisch augmentieren
        for cl_id, (mu, cov, cols) in cluster_params.items():
            if np.random.rand() < p_augment:
                # Ziehe einen neuen Vektor aus der multivariaten Normalverteilung
                new_values = np.random.multivariate_normal(mu, cov)
                # Überschreibe nur die Werte in den entsprechenden Spalten
                for col, val in zip(cols, new_values):
                    new_row[col] = val
        augmented_rows.append(new_row)
    
    augmented_df = pd.DataFrame(augmented_rows)
    
    # Sicherstellen, dass Gruppierungsvariablen den richtigen Datentyp behalten:
    for col in ["Geschlecht_weiblich", "Verletzungsstatus"]:
        if col in augmented_df.columns:
            augmented_df[col] = augmented_df[col].astype(int)
    
    return augmented_df


# ----------------------------------------------------------------
# Funktion: Stratifikation und Augmentation der Trainingsdaten
# ----------------------------------------------------------------
def augment_training_data(X_train, y_train, max_clusters=4, p_augment=0.7, num_new_samples=1):
    """
    Führt zuerst eine Stratifikation des Trainingsdatensatzes nach "Verletzungsstatus"
    und "Geschlecht_weiblich" durch und wendet dann in jeder Subgruppe:
      - eine Clusteranalyse (auf alle übrigen numerischen Features)
      - die Augmentation (Ziehung neuer Samples aus multivariater Normalverteilung)
    an.
    
    num_new_samples gibt an, wie viele augmentierte Samples pro Originalzeile generiert werden.
    """
    # Kombiniere X_train und y_train, damit wir gruppieren können
    df_train = X_train.copy()
    df_train["Verletzungsstatus"] = y_train
    df_train.reset_index(drop=True, inplace=True)
    
    # Stratifikation: Erstelle eine Gruppierung nach (Verletzungsstatus, Geschlecht_weiblich)
    augmented_groups = []
    for key, group_df in df_train.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group_df = group_df.reset_index(drop=True)
        # Führe Clusteranalyse auf dieser Subgruppe durch
        clusters = perform_clustering(group_df, max_clusters=max_clusters)
        # Erzeuge für diese Subgruppe num_new_samples augmentierte Samples pro Zeile
        aug_list = []
        for _ in range(num_new_samples):
            aug = augment_subgroup(group_df, clusters, p_augment=p_augment)
            aug_list.append(aug)
        aug_group = pd.concat(aug_list, axis=0)
        augmented_groups.append(aug_group)
    
    # Vereinige alle augmentierten Subgruppen
    augmented_df = pd.concat(augmented_groups, axis=0).reset_index(drop=True)
    
    # Trenne Zielvariable und Features (hier war "Verletzungsstatus" in df_train enthalten)
    y_aug = augmented_df["Verletzungsstatus"]
    X_aug = augmented_df.drop(columns=["Verletzungsstatus"])
    
    return X_aug, y_aug

# ----------------------------------------------------------------
# Integration in den Trainingssplit (innerer CV) mit Augmentation
# ----------------------------------------------------------------
def inner_cv_with_augmentation(X_train, y_train):
    from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
    from sklearn.linear_model import LogisticRegression

    inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = []

    for train_idx, test_idx in inner_cv.split(X_train, y_train):
        # Kopien der Daten erstellen
        X_inner_train = X_train.iloc[train_idx].copy()
        X_inner_test = X_train.iloc[test_idx].copy()
        y_inner_train, y_inner_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # Normalisierung innerhalb des Folds:
        dummy = "Geschlecht_weiblich"
        cols_to_scale = [col for col in X_inner_train.columns if col != dummy]
        scaler = StandardScaler()
        X_inner_train[cols_to_scale] = scaler.fit_transform(X_inner_train[cols_to_scale])
        X_inner_test[cols_to_scale] = scaler.transform(X_inner_test[cols_to_scale])

        # Augmentation auf den normalisierten Trainingsdaten durchführen:
        X_inner_train_aug, y_inner_train_aug = augment_training_data(
            X_inner_train, y_inner_train, max_clusters=4, p_augment=0.7, num_new_samples=3
        )

        model = LogisticRegression(max_iter=1000)
        # Auch hier wird cross_val_score innerhalb der Trainingsdaten-CV eingesetzt
        score = cross_val_score(model, X_inner_train_aug, y_inner_train_aug, cv=3,
                                scoring="roc_auc", n_jobs=-1).mean()
        scores.append(score)

    avg_score = np.mean(scores)
    #print("Durchschnittliche innere CV-ROC-AUC mit Augmentation:", avg_score)
    return avg_score

#------------------------------------------------------------------------
# Integration in den Trainingssplit (innerer CV) ohne Augmentation
#------------------------------------------------------------------------
def inner_cv_without_augmentation(X_train, y_train):
    from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
    from sklearn.linear_model import LogisticRegression

    inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = []

    for train_idx, test_idx in inner_cv.split(X_train, y_train):
        # Kopien der Daten erstellen, um spätere Änderungen lokal zu halten
        X_inner_train = X_train.iloc[train_idx].copy()
        X_inner_test = X_train.iloc[test_idx].copy()
        y_inner_train, y_inner_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # Normalisierung innerhalb des Folds:
        dummy = "Geschlecht_weiblich"
        cols_to_scale = [col for col in X_inner_train.columns if col != dummy]
        scaler = StandardScaler()
        X_inner_train[cols_to_scale] = scaler.fit_transform(X_inner_train[cols_to_scale])
        X_inner_test[cols_to_scale] = scaler.transform(X_inner_test[cols_to_scale])

        model = LogisticRegression(max_iter=1000)
        # cross_val_score führt hier eine weitere CV auf den inneren Trainingsdaten durch
        score = cross_val_score(model, X_inner_train, y_inner_train, cv=3,
                                scoring="roc_auc", n_jobs=-1).mean()
        scores.append(score)
    
    avg_score = np.mean(scores)
    #print("Durchschnittliche innere CV-ROC-AUC ohne Augmentation:", avg_score)
    return avg_score


In [9]:
# Originaldaten laden
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Basistabelle_ML_neu.xlsx"
df_original = pd.read_excel(file_path)

# Aufteilen in Features und Zielvariable
X_original = df_original.drop(columns=["Verletzungsstatus"])
y_original = df_original["Verletzungsstatus"]

# Vergleich beider Ansätze:
print("=== Vergleich: Innere CV ohne vs. mit Augmentation ===")
score_without = inner_cv_without_augmentation(X_original, y_original)
score_with = inner_cv_with_augmentation(X_original, y_original)

print("\nZusammenfassung der Ergebnisse:")
print("Durchschnittliche innere CV-ROC-AUC ohne Augmentation:", score_without)
print("Durchschnittliche innere CV-ROC-AUC mit Augmentation:", score_with)
print("Differenz:", score_with - score_without)


=== Vergleich: Innere CV ohne vs. mit Augmentation ===
Durchschnittliche innere CV-ROC-AUC ohne Augmentation: 0.9261234109028226
Durchschnittliche innere CV-ROC-AUC mit Augmentation: 0.9763723972941868

Zusammenfassung der Ergebnisse:
ROC-AUC ohne Augmentation: 0.9261234109028226
ROC-AUC mit Augmentation: 0.9763723972941868
Differenz: 0.050248986391364214


Test-Datenaugmentation-Cluster

Test Cluster-Funktion

In [ ]:
def perform_clustering(X_sub, max_clusters=4):
    """
    Führt eine Clusteranalyse auf den numerischen Parametern (außer den
    Gruppierungsvariablen "Verletzungsstatus" und "Geschlecht_weiblich")
    durch und teilt diese in max_clusters Gruppen ein.
    """
    # Wähle alle numerischen Spalten außer "Verletzungsstatus" und "Geschlecht_weiblich"
    cols = [col for col in X_sub.columns if col not in ["Verletzungsstatus", "Geschlecht_weiblich"]]
    X_num = X_sub[cols]
    
    # Berechne die Korrelationsmatrix und transformiere sie in eine Distanzmatrix
    corr = X_num.corr().abs()
    dist = np.clip(1 - corr, a_min=0, a_max=None)  # Clippe negative Werte auf 0
    
    # Konvertiere die Distanzmatrix in ein 1D-Array (upper triangular)
    dists = squareform(dist.values)
    
    # Hierarchische Clusteranalyse (z. B. mit der Ward-Methode)
    Z = linkage(dists, method='ward')
    
    # Führe fcluster aus, um die Spalten in max_clusters Cluster zu unterteilen
    cluster_labels = fcluster(Z, max_clusters, criterion='maxclust')
    
    # Ordne den Spalten Clusterlabels zu
    clusters = {}
    for col, label in zip(cols, cluster_labels):
        clusters.setdefault(label, []).append(col)
    return clusters

Test Subgruppen 

In [ ]:
def augment_subgroup(X_sub, clusters, p_augment=0.7):
    """
    Augmentiert die Daten in X_sub (einen DataFrame für eine Subgruppe)
    für alle in clusters definierten Parametergruppen.
    
    Für jede Zeile und für jede Cluster-Gruppe wird mit Wahrscheinlichkeit p_augment
    ein neuer Wert gezogen – ansonsten wird der Originalwert beibehalten.
    Dabei werden für jede Parametergruppe der Mittelwertvektor und die Kovarianzmatrix
    des gesamten Subdatensatzes berechnet.
    
    Rückgabe: DataFrame mit augmentierten Zeilen (1 augmentiertes Sample pro Originalzeile)
    """
    augmented_rows = []
    
    # Für jede Cluster-Gruppe: Berechne Mittelwert und Kovarianzmatrix
    cluster_params = {}
    for cl_id, cols in clusters.items():
        cluster_data = X_sub[cols]
        mu = cluster_data.mean().values
        # Falls nur ein Feature im Cluster ist, gibt np.cov einen Skalar zurück.
        if len(cols) == 1:
            cov = np.cov(cluster_data.values.flatten(), ddof=0)
            cov = np.atleast_2d(cov)
        else:
            cov = np.cov(cluster_data.values, rowvar=False)
        cluster_params[cl_id] = (mu, cov, cols)
    
    # Gehe jede Zeile (Originaldatensatz) durch
    for idx, row in X_sub.iterrows():
        new_row = row.copy()
        # Für jede Cluster-Gruppe stochastisch augmentieren
        for cl_id, (mu, cov, cols) in cluster_params.items():
            if np.random.rand() < p_augment:
                # Ziehe einen neuen Vektor aus der multivariaten Normalverteilung
                new_values = np.random.multivariate_normal(mu, cov)
                # Überschreibe nur die Werte in den entsprechenden Spalten
                for col, val in zip(cols, new_values):
                    new_row[col] = val
        augmented_rows.append(new_row)
    
    augmented_df = pd.DataFrame(augmented_rows)
    
    # Sicherstellen, dass Gruppierungsvariablen den richtigen Datentyp behalten:
    for col in ["Geschlecht_weiblich", "Verletzungsstatus"]:
        if col in augmented_df.columns:
            augmented_df[col] = augmented_df[col].astype(int)
    
    return augmented_df



Test Daten-Augmentation

In [ ]:
def augment_training_data(X_train, y_train, max_clusters=4, p_augment=0.7, num_new_samples=1):
    """
    Führt zuerst eine Stratifikation des Trainingsdatensatzes nach "Verletzungsstatus"
    und "Geschlecht_weiblich" durch und wendet dann in jeder Subgruppe:
      - eine Clusteranalyse (auf alle übrigen numerischen Features)
      - die Augmentation (Ziehung neuer Samples aus multivariater Normalverteilung)
    an.
    
    num_new_samples gibt an, wie viele augmentierte Samples pro Originalzeile generiert werden.
    """
    # Kombiniere X_train und y_train, damit wir gruppieren können
    df_train = X_train.copy()
    df_train["Verletzungsstatus"] = y_train
    df_train.reset_index(drop=True, inplace=True)
    
    # Stratifikation: Erstelle eine Gruppierung nach (Verletzungsstatus, Geschlecht_weiblich)
    augmented_groups = []
    for key, group_df in df_train.groupby(["Verletzungsstatus", "Geschlecht_weiblich"]):
        group_df = group_df.reset_index(drop=True)
        # Führe Clusteranalyse auf dieser Subgruppe durch
        clusters = perform_clustering(group_df, max_clusters=max_clusters)
        # Erzeuge für diese Subgruppe num_new_samples augmentierte Samples pro Zeile
        aug_list = []
        for _ in range(num_new_samples):
            aug = augment_subgroup(group_df, clusters, p_augment=p_augment)
            aug_list.append(aug)
        aug_group = pd.concat(aug_list, axis=0)
        augmented_groups.append(aug_group)
    
    # Vereinige alle augmentierten Subgruppen
    augmented_df = pd.concat(augmented_groups, axis=0).reset_index(drop=True)
    
    # Trenne Zielvariable und Features (hier war "Verletzungsstatus" in df_train enthalten)
    y_aug = augmented_df["Verletzungsstatus"]
    X_aug = augmented_df.drop(columns=["Verletzungsstatus"])
    
    return X_aug, y_aug


In [ ]:
https://github.com/michi1308/Test_ML.git